In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # You can use other BERT variants like 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # For binary classification

In [2]:
import data_file

15
(15, 384)
15
Vectorstore made


In [ ]:
data_list=data_file.data1
data_list

In [ ]:
data_X=[]
data_Y=[]
for element in data_list:
    data_X.append(element['content'])
    data_Y.append(element['label'])
data_X

In [30]:
def sliding_window_tokenize(text, max_length=512, overlap=256):
    tokens = tokenizer.encode(text)
    windows = []
    for i in range(0, len(tokens), max_length - overlap):
        window = tokens[i:i + max_length]
        windows.append(window)
    return windows
tokenized_cont={}
for i in range(len(data_X)):
    windows = sliding_window_tokenize(data_X[i], max_length=512, overlap=256)
    tokenized_cont[i]=windows

In [46]:
sum=0
for i in range(len(tokenized_cont)):
    sum+=len(tokenized_cont[i])
sum

456

In [54]:
from torch.utils.data import Dataset

# Prepare dataset class
class BinaryClassificationDataset(Dataset):
    def __init__(self, tokenized_windows, labels):
        self.tokenized_windows = tokenized_windows
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_windows)

    def __getitem__(self, idx):
        # Get the tokenized window and corresponding label
        window = self.tokenized_windows[idx]
        label = self.labels[idx]
        
        # Convert to tensor format
        encoding = tokenizer(window, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Remove extra batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        }

# Create the dataset
all_windows = [window for windows in tokenized_cont.values() for window in windows]
all_labels = [label for label in data_Y for _ in range(len(tokenized_cont[0]))]  # Replicate labels for each window

# Initialize dataset
train_dataset = BinaryClassificationDataset(all_windows, all_labels)


In [34]:
type(train_dataset)

__main__.BinaryClassificationDataset

In [55]:
len(train_dataset)

456

In [37]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',            # output directory for model predictions and checkpoints
    num_train_epochs=3,                # number of training epochs
    per_device_train_batch_size=8,     # batch size for training
    per_device_eval_batch_size=8,      # batch size for evaluation
    warmup_steps=500,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir='./logs',              # directory for storing logs
    logging_steps=10,
)

# Setup Trainer
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,         # Pass the dataset
)

# Fine-tune the model
trainer.train()